In [1]:
import contractions
import torch
import unidecode

torch.cuda.empty_cache()

import cupy as cp

# Forceer het vrijmaken van geheugen
cp.get_default_memory_pool().free_all_blocks()

In [ ]:
import torch
import pandas as pd
import spacy
from gliner_spacy.pipeline import GlinerSpacy
from thinc.api import set_gpu_allocator, require_gpu
from unidecode import unidecode
import contractions

# Controleer of GPU beschikbaar is
if torch.cuda.is_available():
    set_gpu_allocator("pytorch")
    require_gpu(0)
    device = "cuda"
    print("GPU wordt gebruikt!")
else:
    device = "cpu"
    print("CPU wordt gebruikt")

torch.cuda.empty_cache()

# Laad Nederlandse en Engelse SpaCy-modellen
print("Laad Nederlandse en Engelse SpaCy-modellen...")
nl_model = spacy.load('nl_core_news_sm')
en_model = spacy.load('en_core_web_sm')

# Configuratie voor GLiNER
custom_spacy_config = {
    "gliner_model": "urchade/gliner_mediumv2.1",
    "chunk_size": 250,
    "labels": ["persoon", "datum", "organisatie", "gebeurtenis"],
    "style": "ent",
    "threshold": 0.4,
    "map_location": device
}

# Voeg de GLiNER pipe toe aan de modellen
nl_model.add_pipe('gliner_spacy', last=True, config=custom_spacy_config)
en_model.add_pipe('gliner_spacy', last=True, config=custom_spacy_config)

# CSV-bestanden
csv_path = '/home/nena-meijer/PyCharmMiscProject/dataset/VWS_subset/4-VWS_documents_no_empties.csv'
output_csv_path = '/home/nena-meijer/PyCharmMiscProject/dataset/VWS_subset/5-VWS_documents_NER_nl_labels.csv'

test_mode = False  # Zet op False om de hele dataset te verwerken
num_test_rows = 10  # Aantal rijen in testmodus
chunksize = 300 if not test_mode else num_test_rows

df_result = pd.DataFrame()

excluded_persons = {"ik", "je", "jij", "u", "wij", "we", "zij", "hij", "haar", "hem", "hun", "mij", "me", "jou", "jullie", "your", "you", "his", "her", "they", "them"}
def filter_entities(persons):
    """Verwijder ongewenste woorden uit de lijst met 'persons'-entiteiten."""
    return [p for p in persons if p.lower() not in excluded_persons]

def extract_entities(text, lang):
    if not isinstance(text, str) or not text.strip():
        return [], [], [], [], []

    nlp_model = nl_model if lang == 'nl' else en_model

    if 'gliner_spacy' not in nlp_model.pipe_names:
        print("⚠️ GLiNER niet meer in pipeline! Model herinitialiseren...")
        nlp_model.add_pipe('gliner_spacy', last=True, config=custom_spacy_config)

    doc = nlp_model(text)

    persons = [ent.text for ent in doc.ents if ent.label_ == 'persoon']
    dates = [ent.text for ent in doc.ents if ent.label_ == 'datum']
    organisations = [ent.text for ent in doc.ents if ent.label_ == 'organisatie']
    events = [ent.text for ent in doc.ents if ent.label_ == 'gebeurtenis']

    del doc
    torch.cuda.empty_cache()

    persons = filter_entities(persons)

    return persons, dates, organisations, events

for batch_num, chunk in enumerate(pd.read_csv(csv_path, chunksize=chunksize), 1):
    print(f"\nVerwerken van batch {batch_num}...")
    result = []

    chunk.reset_index(drop=True, inplace=True)

    if 'document_text' not in chunk.columns or 'document_language' not in chunk.columns:
        raise KeyError("De vereiste kolommen 'document_text' of 'document_language' ontbreken in de dataset.")

    for idx, row in chunk.iterrows():
        if idx % 100 == 0:
            print(f"Verwerkte {idx} rijen van de batch.")

        try:
            document_text = str(row['document_text'])
            document_text = contractions.fix(unidecode(row['document_text'])).strip().replace('\n', '')
            entities = extract_entities(document_text, row['document_language'])
            result.append(entities)
        except Exception as e:
            print(f"Fout bij rij {idx}, {document_text}: {e}")
            raise

    chunk[['NER_persons', 'NER_dates', 'NER_orgs', 'NER_events']] = pd.DataFrame(result, columns=['NER_persons', 'NER_dates', 'NER_orgs', 'NER_events'])

    if test_mode:
        display(chunk[['NER_persons', 'NER_dates', 'NER_orgs', 'NER_events']])
        break  # Stop na de testbatch

    chunk.to_csv(f"/home/nena-meijer/PyCharmMiscProject/dataset/VWS_subset/NER_batches/NER_batch_{batch_num}.csv", index=False)
    df_result.to_csv(output_csv_path, index=False)

torch.cuda.empty_cache()
print("\nVerwerking voltooid.")


In [ ]:
# gliner test

import spacy
from gliner_spacy.pipeline import GlinerSpacy

nlp = spacy.load('nl_core_news_lg')
nlp.add_pipe("gliner_spacy")
text = "611783 ee iow: @minvws.nl from: } sent: tue 10/26/2021 8:13:10 am subject: re: conceptartikel voor vws#net over wobverzoeken corona received: tue 10/26/2021 8:13:11 am hoi allen, eere verwerkt de opmerkingen van eee in het voorliggende hoevertelikhetthuisinterview en verwijst alvast naar vws#dianovember, waarin we een achtergrondartikel plaatsen over de nieuwe programmadirectie met 0.a. antwoord op de vragen die [&gY=erecht stelt. later deze week stort ich op dat vws#diaartikel ik stuur jullie begin van de middag de 'eindversie' van het huidige vws#netinterview zodat jullie er een laatste (goedkeurende) blik op kunnen werpen. het streven is om dit in de loop van de middag te publiceren op vws#net. iedereen happy zo? met vriendelijke groet, 5.1.2e parnassusplein 5 | 2511 vx | den haag | kamer postbus 20350 | 2500 ej | den haag t070 minvws.nt http://www. rijksoverhe nl nederland gezond en wel. dubbel 611783 611783"

doc = nlp(text)
for ent in doc.ents:
    print(ent.text, '-->', ent.label_)


In [ ]:
# NER BERTje/BERT

import pandas as pd
from transformers import pipeline

# Laad de pipeline voor NER met BERT (Nederlands en Engels)
nl_ner_pipeline = pipeline("ner", model="GroNLP/bert-base-dutch-cased")
en_ner_pipeline = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english")

# Laad de CSV met de documenttekst
csv_path = '/home/nena-meijer/PyCharmMiscProject/dataset/VWS/8-VWS_documents_normalized-FINAL.csv'
df = pd.read_csv(csv_path)

# Functie om NER uit te voeren en datums en namen te extraheren
def extract_dates_and_names(text, lang):
    if not isinstance(text, str):  # Controleer of de tekst geldig is
        return "", ""  # Geen datums of namen

    # Kies het juiste model op basis van de taal
    ner_pipeline = nl_ner_pipeline if lang == 'nl' else en_ner_pipeline

    # Pipeline om NER uit te voeren
    ner_results = ner_pipeline(text)

    print(f"NER results for text: {ner_results}")  # Voeg deze regel toe om te kijken naar de resultaten

    # Filter voor datums en namen
    dates = []
    names = []

    for entity in ner_results:
        if entity['entity'] == 'B-DATE' or entity['entity'] == 'I-DATE':
            dates.append(entity['word'])
        if entity['entity'] == 'B-PER' or entity['entity'] == 'I-PER':
            names.append(entity['word'])

    # Zet de datums en namen om naar een enkele string gescheiden door een puntkomma
    dates_str = "; ".join(dates)
    names_str = "; ".join(names)

    return dates_str, names_str


# Verwerk alleen de eerste 10 rijen
df_subset = df.iloc[:10].copy()  # Kopie maken om de originele dataframe niet te overschrijven
df_subset['NER_dates'], df_subset['NER_names'] = zip(*df_subset.apply(lambda row: extract_dates_and_names(row['document_text'], row['document_language']), axis=1))

# Bekijk de resultaten
df_subset[['document_text', 'NER_dates', 'NER_names']]


In [ ]:
# datums regex

import pandas as pd
import re
import dateparser

# Laad de CSV met de documenttekst
csv_path = '/home/nena-meijer/PyCharmMiscProject/dataset/VWS/9-VWS_documents_with_ministries.csv'
df = pd.read_csv(csv_path)

# Reguliere expressies voor extra datumformaten
date_patterns = [
    r'\b\d{1,2}/\d{1,2}/\d{4}\b',         # 26/4/2020, 04/26/2020
    r'\b\d{1,2}-\d{1,2}-\d{4}\b',         # 26-4-2020, 04-26-2020
    r'\b\d{4}/\d{1,2}/\d{1,2}\b',         # 2020/04/26
    r'\b\d{4}-\d{1,2}-\d{1,2}\b',         # 2020-04-26
    r'\b\d{1,2} [a-zA-Zé]+\.* \d{4}\b',   # 26 april 2020, 26 apr. 2020, 26 Apr. 2020
    r'\b[a-zA-Zé]+ \d{1,2}, \d{4}\b',     # April 26, 2020 (Engelse volgorde)
    r'\b\d{1,2}(?:e|ste)? [a-zA-Zé]+ \d{4}\b',  # 26e april 2020, 26ste april 2020
]

def extract_dates_with_regex(text):
    dates = []
    for pattern in date_patterns:
        matches = re.findall(pattern, text)
        dates.extend(matches)
    return dates

def normalize_date(date_str):
    # Probeer eerst de Engelse datuminstelling voor MM/DD/YYYY (MDY)
    parsed_date = dateparser.parse(date_str, languages=['en'], settings={'DATE_ORDER': 'MDY'})

    # Als dat niet werkt, probeer de Nederlandse instelling voor DD/MM/YYYY (DMY)
    if not parsed_date:
        parsed_date = dateparser.parse(date_str, languages=['nl'], settings={'DATE_ORDER': 'DMY'})

    # Controleer of de datum geldig is en het jaartal in het juiste bereik valt
    if parsed_date and 2010 <= parsed_date.year < 2030:
        return parsed_date.strftime('%Y-%m-%d')
    return None

def extract_dates_with_regex_only(text):
    if not isinstance(text, str):
        return ""

    regex_dates = extract_dates_with_regex(text)

    seen_dates = set()
    all_dates = []

    for date in regex_dates:
        normalized_date = normalize_date(date)
        if normalized_date and normalized_date not in seen_dates:
            seen_dates.add(normalized_date)
            all_dates.append(normalized_date)

    return "; ".join(all_dates)

# Verwerk de volledige DataFrame met alleen regex
df['NER_dates'] = df['document_text'].apply(extract_dates_with_regex_only)

# Sla de resultaten op in een nieuwe CSV
output_csv_path = '/dataset/VWS/9-VWS_documents_dates&mins_regex.csv'
df.to_csv(output_csv_path, index=False)

# Optioneel: Bekijk de eerste paar rijen van het resultaat
df[['document_text', 'NER_dates']].head()


In [ ]:
# precision datums regex

import pandas as pd

# Laad de CSV in
df = pd.read_csv("/dataset/VWS/9-VWS_documents_dates&mins_regex.csv")

# Zorg ervoor dat de kolom NER_dates wordt omgezet naar een lijst van datums
df["NER_dates"] = df["NER_dates"].apply(lambda x: x.split(";") if isinstance(x, str) else [])

# Precision: Hoe vaak is de eerste NER-date gelijk aan document_date?
df["first_match"] = df.apply(lambda row: row["NER_dates"][0] == row["document_date"] if row["NER_dates"] else False, axis=1)
precision = df["first_match"].mean()

# Algemene aanwezigheid: Hoe vaak komt document_date ergens in NER_dates voor?
df["any_match"] = df.apply(lambda row: row["document_date"] in row["NER_dates"] if row["NER_dates"] else False, axis=1)
general_presence = df["any_match"].mean()

print(f"Precision (eerste datum matcht): {precision:.2%}")
print(f"Algemene aanwezigheid (document_date komt ergens voor): {general_presence:.2%}")
